In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [2]:
from pathlib import Path

In [3]:
dPath = Path("../docs/dumps")

In [4]:
import pickle

In [5]:
with open(dPath / "train_data.pkl", 'rb') as filename:
    train_data = pickle.load(filename)

In [6]:
with open(dPath / "valid_data.pkl", 'rb') as filename:
    valid_data = pickle.load(filename)

In [7]:
with open(dPath / "not_running.pkl", 'rb') as filename:
    not_running = pickle.load(filename)

In [8]:
X_train = train_data.drop("Detected", axis=1)

In [9]:
y_train = train_data.Detected

In [10]:
X_valid = valid_data.drop("Detected", axis=1)

In [11]:
y_valid = valid_data.Detected

In [12]:
with open(dPath / "rf_exp_05_names.pkl", 'rb') as filename:
    names = pickle.load(filename)

In [13]:
X_train = X_train[names] 

In [14]:
X_valid = X_valid[names]

In [15]:
X_train.head()

,numExecuted,MutatorClass,numAssertInTC,numTestCover,numAssertInTM,ppavcc,cchalsteadCumulativeBugs,cchalsteadCumulativeVolume,ppRVF,ppnumberOfMethods,...,cchalsteadEffort,ccmaintainabilityIndexNC,mmhalsteadVocabulary,pptcc,mmexternalMethodsCalled,ppinstability,ppmaxcc,mmhalsteadEffort,mmvariablesReferenced,ccunweightedClassSize
0,14,2,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
1,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
2,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
3,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
4,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0


In [16]:
from imblearn.over_sampling import ADASYN

In [17]:
sm = ADASYN(random_state=42, n_jobs=-1,  n_neighbors=5)

In [18]:
%time X_res, y_res = sm.fit_resample(X_train, y_train)

Wall time: 51.5 s


In [19]:
X_train_not_running = not_running.drop("Detected", axis=1)

In [20]:
y_train_not_running = not_running.Detected

In [21]:
X_train_not_running = X_train_not_running[names]

In [22]:
X_train = pd.concat([X_res,X_train_not_running,X_valid], ignore_index=True,axis=0)

In [23]:
y_train = pd.concat([y_res,y_train_not_running,y_valid], ignore_index=True,axis=0)

In [24]:
y_train.sum()/y_train.shape[0]

0.48736511651307735

In [25]:
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble  import HistGradientBoostingClassifier

In [26]:
models = []
np.random.seed(42)

for i in tqdm(range(50)):
    
    hgb = HistGradientBoostingClassifier(random_state=np.random.randint(2 ** 10), max_iter=3600, validation_fraction=None)

    hgb.fit(X_train, y_train)
    models.append(hgb)

C:\Users\Alireza\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [27]:
with open(dPath / "hgb_exp_04_stacking.pkl", 'wb') as filename:
    pickle.dump(models,filename)